In [1]:
# https://github.com/scipy/scipy/issues/7151
# https://apps.dtic.mil/sti/pdfs/AD1004183.pdf
# https://www.codeproject.com/Articles/21282/Compute-Permanent-of-a-Matrix-with-Ryser-s-Algorit

# https://rosettacode.org/wiki/Determinant_and_permanent
# https://codegolf.stackexchange.com/questions/97060/calculate-the-permanent-as-quickly-as-possible

# https://stackoverflow.com/questions/38738835/generating-gray-codes
# https://qiita.com/b1ueskydragon/items/75cfee42541ea723080c

# https://qiita.com/phdax/items/3064de264c7933bab2f5
# https://web.archive.org/web/20190108235115/https://www.hackersdelight.org/hdcodetxt/pop.c.txt
# http://graphics.stanford.edu/~seander/bithacks.html#CountBitsSetParallel
# https://stackoverflow.com/questions/9829578/fast-way-of-counting-non-zero-bits-in-positive-integer

# https://stackoverflow.com/questions/22227595/convert-integer-to-binary-array-with-suitable-padding

In [2]:
import numpy as np

from itertools import permutations
from operator import mul
from math import fsum
from functools import reduce

import scipy.special as spsp

import time

from numba import jit

In [3]:
# https://github.com/XanaduAI/thewalrus/blob/master/thewalrus/_permanent.py

@jit(nopython=True)
def perm_bbfg(M):  # pragma: no cover
    """
    Returns the permanent of a matrix using the bbfg formula in Gray ordering
    The code is a re-implementation from a Python 2 code found in
    `Permanent code golf
    <https://codegolf.stackexchange.com/questions/97060/calculate-the-permanent-as-quickly-as-possible>`_
    using Numba.
    Args:
        M (array) : a square array.
    Returns:
        float or complex: the permanent of a matrix ``M``
    """

    n = len(M)
    if n == 0:
        return M.dtype.type(1.0)
    row_comb = np.sum(M, 0)
    total = 0
    old_gray = 0
    sign = +1
    binary_power_dict = np.array([2**i for i in range(n)])
    num_loops = 2 ** (n - 1)
    for bin_index in range(1, num_loops + 1):
        reduced = np.prod(row_comb)
        total += sign * reduced
        new_gray = bin_index ^ (bin_index // 2)
        gray_diff = old_gray ^ new_gray
        gray_diff_index = np.searchsorted(binary_power_dict,gray_diff)
        new_vector = M[gray_diff_index]
        direction = 2 * ((old_gray > new_gray) - (old_gray < new_gray))
        for i in range(n):
            row_comb[i] += new_vector[i] * direction
        sign = -sign
        old_gray = new_gray
    return total / num_loops

In [4]:
for L in range(4,30):
    a = np.ones((L,L),dtype=np.float64)
    #print(a)
    print("L:",L)
    start = time.time()
    print(perm_bbfg(a))
    end = time.time()
    print("time:",end-start)
    print("")

L: 4
24.0
time: 1.1851489543914795

L: 5
120.0
time: 1.6927719116210938e-05

L: 6
720.0
time: 1.0013580322265625e-05

L: 7
5040.0
time: 1.0013580322265625e-05

L: 8
40320.0
time: 1.0967254638671875e-05

L: 9
362880.0
time: 1.1920928955078125e-05

L: 10
3628800.0
time: 1.6927719116210938e-05

L: 11
39916800.0
time: 2.6226043701171875e-05

L: 12
479001600.0
time: 4.792213439941406e-05

L: 13
6227020800.0
time: 9.489059448242188e-05

L: 14
87178291200.0
time: 0.00020575523376464844

L: 15
1307674367999.6401
time: 0.0003948211669921875

L: 16
20922789888000.0
time: 0.0007722377777099609

L: 17
355687428096240.0
time: 0.0016319751739501953

L: 18
6402373705657936.0
time: 0.0036029815673828125

L: 19
1.2164510040934886e+17
time: 0.008055925369262695

L: 20
2.4329020081668424e+18
time: 0.01995110511779785

L: 21
5.109094217405589e+19
time: 0.03595876693725586

L: 22
1.1240007277778578e+21
time: 0.07564115524291992

L: 23
2.5852016740764907e+22
time: 0.15198493003845215

L: 24
6.20448401791587